In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda:0"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

In [5]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.699184 auc:0.4962
epoch:  20 loss:0.349559 auc:0.9150
epoch:  40 loss:0.323466 auc:0.9237
epoch:  60 loss:0.299554 auc:0.9290
epoch:  80 loss:0.274975 auc:0.9339
epoch: 100 loss:0.255808 auc:0.9373
epoch: 120 loss:0.242727 auc:0.9395
epoch: 140 loss:0.234441 auc:0.9410
epoch: 160 loss:0.228984 auc:0.9418
epoch: 180 loss:0.224676 auc:0.9424
epoch: 200 loss:0.223814 auc:0.9420
epoch: 220 loss:0.219521 auc:0.9433
epoch: 240 loss:0.216629 auc:0.9437
epoch: 260 loss:0.215944 auc:0.9436
epoch: 280 loss:0.214392 auc:0.9438
epoch: 300 loss:0.213097 auc:0.9440
epoch: 320 loss:0.212173 auc:0.9440
epoch: 340 loss:0.214010 auc:0.9432
epoch: 360 loss:0.210489 auc:0.9440
epoch: 380 loss:0.216959 auc:0.9409
epoch: 400 loss:0.210947 auc:0.9439
epoch: 420 loss:0.208557 auc:0.9441
epoch: 440 loss:0.208458 auc:0.9440
epoch: 460 loss:0.207380 auc:0.9440
epoch: 480 loss:0.208000 auc:0.9441
epoch: 500 loss:0.206773 auc:0.9439
epoch: 520 loss:0.206927 auc:0.9438
epoch: 540 loss:0.213053 auc

In [6]:
true_datas.to_csv("true_ctrp.csv")
predict_datas.to_csv("pred_ctrp.csv")